In [2]:
import sys
sys.executable

'/opt/anaconda3/bin/python3'

In [5]:
!/opt/anaconda3/bin/pip install sklearn --user

  Using cached https://files.pythonhosted.org/packages/19/96/8034e350d4550748277e514d0d6d91bdd36be19e6c5f40b8af0d74cb0c84/scikit_learn-0.22-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/28/5c/cf6a2b65a321c4a209efcdf64c2689efae2cb62661f8f6f4bb28547cf1bf/joblib-0.14.1-py2.py3-none-any.whl


In [3]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np

/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passi

In [7]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2068544266075719452
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 13574654590561000430
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 9558320008213973088
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:1"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 734147575090736606
physical_device_desc: "device: XLA_GPU device"
]



## Dataflow

Dataflow - концепция программирования: программа или модель представляется в форме направленного графа (_вычислений_).

Такой подход обладает следущими преимуществами: 
* Простота параллелизации программы: по графу легко понять, какие операции можно выполнять одновременно
* Распределенные вычисления (кластеры видеокарт, CPU, TPU)
* Компиляция графа: создается оптимизированный код для вычислений
* Граф вычислений - универсальное представление, портируемое между различными языками и платформами

### Мы будет говорить об интерфейсе на python

### Граф

В python представлен классом `tf.Graph`

У графа есть следующие "основные" составляющие:
* структура графа - ребра и узлы (edges и nodes)
* коллекции, связанные с графом (подробности далее)


#### Пример графа
<img src="files/img/simple_graph.png">

### Узлы и ребра

* Узлы графа - это операции `tf.Operation`
* Ребра графа - значения, представленные наследниками класса `tf.Tensor`


#### Добавление значений в граф

* Базовый "кирпичик" - функция `tf.constant(x)`, или операция, всегда возвращающая x. 

> Например, операция `tf.constant(13)` создает `tf.Tensor` (ребро) со значением $13$

* Другой базовый элемент - `tf.Variable(x)`, создающий узел, в котором хранится _изменяемое_ значение. Это может быть полезно, например, при обучении модели: в переменной будут храниться веса модели. 
* Над тензорами можно проводить операции, создавая новые узлы.
* Для оптимизации нужно вызвать `tf.train.Optimizer.minimize` - и ко всем операциям в графе будут добавлены операции (и связанные с ними тензоры), вычисляющие градиенты.


#### Пример



In [8]:
a = tf.constant(2)  # Создаем узлы графа c константами
b = tf.constant(2)  
c = a + b  # Складываем значения - создаем новый узел

print(a)
print(b)
print(c)

Tensor("Const:0", shape=(), dtype=int32)
Tensor("Const_1:0", shape=(), dtype=int32)
Tensor("add:0", shape=(), dtype=int32)


In [9]:
print(a)
print(b)
print(c)

Tensor("Const:0", shape=(), dtype=int32)
Tensor("Const_1:0", shape=(), dtype=int32)
Tensor("add:0", shape=(), dtype=int32)


## Создаем сессию и сохраняем результаты 

> Скоро разберемся с сессиями и tensorboard

In [10]:
tf.get_default_graph()

In [11]:
with tf.Session() as sess:
    writer = tf.summary.FileWriter('logs', sess.graph)  # logs - имя директории, где будут храниться результаты
    print(sess.run(a))  # Получаем результат вычислений в сессии
    writer.close()

2


### Конструкция with

```
with Sometging() as s:
    s.do_something()
```

* Создается контекст, в котором будет использоваться `s`.
* В `s` записывается значение, вызываемое методом `Something.__enter__` 
* Внутри блока with доступны переменные из скоупа выше.
* После завершения блока доступны переменные и объекты, созданные или измененные внутри блока. В том числе, и объект `s`
* Но! Объект `s` закрыт методом `Something.__exit__`, так что, скорее всего, к нему нельзя обратиться.

In [12]:
with tf.Session() as sess:
    writer = tf.summary.FileWriter('logs', sess.graph)  # logs - имя директории, где будут храниться результаты
    print(sess.run([a, b, c]))  # Получаем результат вычислений в сессии
    writer.close()

[2, 2, 4]


### Пример взаимодейтсвия с закрытой сессией

> RuntimeError: Attempted to use a closed Session.

In [14]:
sess.run([a, b, c])

RuntimeError: Attempted to use a closed Session.



#### Обозначения
<img src="files/img/legend.png" width="400">




### tf.Session

> "Просто так" значения в графе не вычисляются, нужно создавать и запускать сессию, чтобы получить результаты

Несколько фактов:
* Класс `tf.Session`
* Сайт tensorflow сообщает, что "сессия инкапсулирует окружение, в котором выполняются `tf.Operation` и вычисляются значения `tf.Tensor`"
* Можно активировать eager mode, в котором вычисления осуществляются "на лету", тогда `tf.Session` не нужен
* В tensorflow 2.0 от `tf.Session` отказались 

### Почему отказались от сессий? 

Сигнатура класса: 
> `tf.Session(target='', graph=None, config=None)`

* Если явно не передавать `graph`, будет вызван граф по умолчанию: `tf.get_default_graph()`. Помним, что "Explicit is better than implicit".
* Необходимо следить за состоянием сессии и не забывать закрывать/открывать ее на стороне пользователя. 
* Плохо сочетается с eager mode. 





In [21]:
a = tf.constant(1)
b = tf.constant(2)
c = a + b

sess = tf.Session()
sess.run(c)

3


Чего не хватает?

Сессия может "захватывать" ресурсы, после завершения вычислений их надо освободить. Есть два способа:

> * `sess.close()` 
* `with tf.Session() as sess: ...`
* Второй способ закрывает сессию автоматически и предпочтительнее


In [22]:
sess.close()

try:
    print(sess.run(c))
except RuntimeError:
    print("Session is closed")

Session is closed


In [23]:
sess.run(c)

RuntimeError: Attempted to use a closed Session.